In [37]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPool2D

from sklearn.model_selection import train_test_split

import os

In [38]:
# Function to check if ModelCheckpoint callback is used and load the latest checkpoint
def load_latest_checkpoint(model, callbacks):
    # Check if ModelCheckpoint callback is present
    checkpoint_callback = None
    for callback in callbacks:
        if isinstance(callback, tf.keras.callbacks.ModelCheckpoint):
            checkpoint_callback = callback
            break
    
    if checkpoint_callback is not None:
        checkpoint_dir = os.path.dirname(checkpoint_callback.filepath)
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        
        if latest_checkpoint:
            print(f"Loading weights from the latest checkpoint: {latest_checkpoint}")
            model.load_weights(latest_checkpoint)
        else:
            print("No checkpoint found.")
    else:
        print("No ModelCheckpoint callback found.")

In [39]:
#Load the data
(X_train_unscaled, y_train), (X_test_unscaled, y_test) = cifar10.load_data()

print(f"X_train shape: {np.shape(X_train_unscaled)}")
print(f"y_train shape: {np.shape(y_train)}")
print(f"X_test shape: {np.shape(X_test_unscaled)}")
print(f"y_test shape: {np.shape(y_test)}")

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [40]:
#scale the normalize the data and convert targets to one hot encoding
X_train_valid, X_test = X_train_unscaled/255.0, X_test_unscaled/255.0

y_cat_train_valid = to_categorical(y_train)
y_cat_test = to_categorical(y_test)

#split the training dataset again to produce a validation set
X_train, X_valid, y_cat_train, y_cat_valid = train_test_split(X_train_valid, y_cat_train_valid, test_size=0.25, random_state=42)

In [41]:
#build the model
INPUT_SHAPE = (32,32,3)
KERNEL_SIZE = (3,3)
model = Sequential()

#common to increase number of filters as we add convolutional layers (pooling reduces by 2 so we double filters)

#conv layers
model.add(Conv2D(filters=32, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
#pooling
model.add(MaxPool2D(pool_size=(2,2)))
#droput
model.add(Dropout(0.25))

#conv layers
model.add(Conv2D(filters=64, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
#pooling
model.add(MaxPool2D(pool_size=(2,2)))
#droput
model.add(Dropout(0.25))

#conv layers
model.add(Conv2D(filters=128, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))

model.add(BatchNormalization())
#pooling
model.add(MaxPool2D(pool_size=(2,2)))
#droput
model.add(Dropout(0.25))

#fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

#callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    './checkpoints/model_checkpoint.ckpt', 
    save_weights_only=True, 
    save_best_only=True
)

# Example callbacks list
callbacks = [checkpoint_callback]

# load the last checkpoint
load_latest_checkpoint(model, callbacks)

#define metrics
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

Loading weights from the latest checkpoint: ./checkpoints\model_checkpoint.ckpt


In [42]:
#show the summary of the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_24 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_25 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_25 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                      

In [43]:
#train the model
num_epochs = 3
model.fit(X_train, y_cat_train, 
          epochs=num_epochs, 
          validation_data=(X_valid,y_cat_valid), 
          batch_size=32,
          callbacks=callbacks)

Epoch 1/3
1172/1172 [==============================] - 370s 314ms/step - loss: 0.5451 - accuracy: 0.8128 - precision: 0.8746 - recall: 0.7574 - val_loss: 0.6447 - val_accuracy: 0.7897 - val_precision: 0.8519 - val_recall: 0.7394
Epoch 2/3
1172/1172 [==============================] - 364s 310ms/step - loss: 0.5178 - accuracy: 0.8249 - precision: 0.8819 - recall: 0.7731 - val_loss: 0.6580 - val_accuracy: 0.7906 - val_precision: 0.8402 - val_recall: 0.7555
Epoch 3/3
1172/1172 [==============================] - 272s 232ms/step - loss: 0.4932 - accuracy: 0.8321 - precision: 0.8844 - recall: 0.7851 - val_loss: 0.5719 - val_accuracy: 0.8184 - val_precision: 0.8668 - val_recall: 0.7734


In [44]:
#evaluate the model on the test set
test_results = model.evaluate(X_test, y_cat_test)
loss = test_results[0]
accuracy = test_results[1]
precision = test_results[2]
recall = test_results[3]

print(f'accuracy: {accuracy}')
print(f'precision: {precision}')
print(f'recall: {recall}')

313/313 [==============================] - 3s 10ms/step - loss: 0.6065 - accuracy: 0.8083 - precision: 0.8612 - recall: 0.7676
accuracy: 0.8083000183105469
precision: 0.8612139821052551
recall: 0.7675999999046326


In [45]:
#save the model and the test data for 
model.save('cifar10_CNN.h5')